In [1]:
import torch
import torch.nn as nn

# Multiple Input Channels

In [3]:
def conv2d(x, kernel):
    y_h, y_w = x.shape[0] - kernel.shape[0] + 1, x.shape[1] - kernel.shape[1] + 1
    k_h, k_w = kernel.shape[0], kernel.shape[1]
    y = torch.zeros(y_h, y_w)
#     pdb.set_trace()
    for i in range(y_h):
        for j in range(y_w):
#             pdb.set_trace()
            y[i, j] = (x[i:i+k_h, j:j+k_w] * kernel).sum()
            
    return y

In [15]:
def multi_in_conv2d(data, kernel):
#     assert data.shape[0] == kernel.shape[0]
#     output = torch.zeros(data.shape[1] - kernel.shape[1] + 1, data.shape[2] - kernel.shape[2] + 1)
#     for i in range(data.shape[0]):
#         output += conv2d(data[i], kernel[i])
        
#     return output
    return sum([conv2d(x, k) for x, k in zip(data, kernel)])

In [16]:
X = torch.tensor([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
                  [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = torch.tensor([[[0, 1], [2, 3]],
                  [[1, 2], [3, 4]]])

multi_in_conv2d(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

# Multiple Output Channels

In [24]:
def multi_out_conv(data, kernel):
    return torch.stack([multi_in_conv2d(data, sub_k) for sub_k in kernel])

In [26]:
K2 = torch.stack([K,K+1, K+2])
K2.shape, multi_out_conv(X, K2).shape, multi_out_conv(X, K2)

(torch.Size([3, 2, 2, 2]),
 torch.Size([3, 2, 2]),
 tensor([[[ 56.,  72.],
          [104., 120.]],
 
         [[ 76., 100.],
          [148., 172.]],
 
         [[ 96., 128.],
          [192., 224.]]]))

# 1×1  Convolutional Layer

### like fully connected layer

In [27]:
X = torch.randn(size=(3, 3, 3))
K = torch.randn(size=(2, 3, 1, 1))

In [29]:
def fc_layer(data, kernel):
    c_i, h, w, c_o = data.shape[0], data.shape[1], data.shape[2], kernel.shape[0]
    data = data.reshape(c_i, -1)
    kernel = kernel.reshape(c_o, c_i)
    return torch.mm(kernel, data).reshape(c_o, h, w)

In [31]:
fc_layer(X, K), fc_layer(X, K).shape

(tensor([[[-0.4097,  1.1755, -0.3172],
          [ 1.6547, -1.2442,  1.3221],
          [-1.3738,  0.0761, -0.4492]],
 
         [[-1.3356, -2.6595,  2.1876],
          [-0.6619, -0.7226, -2.3644],
          [ 2.4354,  1.0943,  0.9356]]]),
 torch.Size([2, 3, 3]))

In [32]:
multi_out_conv(X, K), fc_layer(X, K).shape

(tensor([[[-0.4097,  1.1755, -0.3172],
          [ 1.6547, -1.2442,  1.3221],
          [-1.3738,  0.0761, -0.4492]],
 
         [[-1.3356, -2.6595,  2.1876],
          [-0.6619, -0.7226, -2.3644],
          [ 2.4354,  1.0943,  0.9356]]]),
 torch.Size([2, 3, 3]))